<a href="https://colab.research.google.com/github/tomonari-masada/course2021-stats1/blob/main/binomial_ML_estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 二項分布のパラメータの最尤推定をPyTorchで

* PyTorchのインポート

In [1]:
import torch

## 二項分布の作り方を確認

* 総試行回数と、パラメータを指定。

* まずは$\phi=0$と設定して、100回試行の場合の52という回数の対数尤度を求める。
 * 尤度はほぼゼロになる。つまり、$\phi=0$のとき、52回という回数はほぼありえない、ということ。

In [2]:
phi = torch.tensor([0.])
m = torch.distributions.binomial.Binomial(100, probs=phi)

In [3]:
m.log_prob(torch.tensor([52]))

tensor([-762.2994])

In [4]:
m.log_prob(torch.tensor([52])).exp()

tensor([0.])

* 次に$\phi=0.5$と設定して、100回試行の場合の52という回数の対数尤度を求める。
 * 尤度は大きくなる。つまり、$\phi=0.5$のときなら、52回という回数はかなりありえる、ということ。

In [5]:
phi = torch.tensor([0.5], requires_grad=True)
m = torch.distributions.binomial.Binomial(100, probs=phi)

In [6]:
m.log_prob(torch.tensor([52]))

tensor([-2.6101], grad_fn=<SubBackward0>)

In [7]:
m.log_prob(torch.tensor([52])).exp()

tensor([0.0735], grad_fn=<ExpBackward>)

## 対数尤度最大化をPyTorchで実装

* 二項分布のパラメータは、微分可能なテンソルとして作っておく。
* そして、そのパラメータを更新するoptimizerを作る。
 * optimizerは、簡単にSGDにしておく。
 * 学習率は小さくしておかないと、最適化がうまくいかないかも。

In [8]:
phi = torch.tensor([0.5], requires_grad=True)
optimizer = torch.optim.SGD([phi], lr=0.0001)

* 1000回のイテレーションで、negative log likelihoodを最小化する。
 * PyTorchでは最小化の計算しかできないので、最大化したいときは、マイナスを付けたものを最小化する。

In [9]:
for i in range(1000):
  optimizer.zero_grad()
  m = torch.distributions.binomial.Binomial(100, probs=phi)
  loss = - m.log_prob(torch.tensor([52]))
  loss.backward()
  optimizer.step()
  if (i + 1) % 100 == 0:
    print(i+1, phi.item())

100 0.51966392993927
200 0.5199943780899048
300 0.5199993252754211
400 0.5199993252754211
500 0.5199993252754211
600 0.5199993252754211
700 0.5199993252754211
800 0.5199993252754211
900 0.5199993252754211
1000 0.5199993252754211


* ちゃんと0.52という答えが出ている！